# Python

In this section we will be examining how you can use YAML tables and views to structure the data you receive from a router using PyEZ.  Some people find this easier to handle than native XML.
<br><br>
We will also take a look at how you can push configuration to a router using Junos PyEZ.  This example will also explore how to handle exceptions.

## Junos PyEZ tables & Views

The Junos PyEZ libraries come with lots of tables and views built in.  You can use these to structure your data or you can use them as templates to build your own. 

(Junos PyEZ libraries might be installed in a different location on your system than the one given in the example below.  You can find where python is loading the files from by loading the python interpreter in verbose mode e.g.:```python3 -v``` and then importing the table e.g. ```from jnpr.junos.op.arp import ArpTable```)


In [15]:
from jnpr.junos import Device
from jnpr.junos.op.arp import ArpTable

ModuleNotFoundError: No module named 'jnpr'

In [11]:
!ls ~/Works/Practice/DEVOPS/Documents/JNCIS/source/venv/lib/python3.9/site-packages/jnpr/junos/op | grep yml | xargs

arp.yml bfd.yml bgp.yml ccc.yml ddos.yml elsethernetswitchingtable.yml ethernetswitchingtable.yml ethport.yml fpc.yml idpattacks.yml intopticdiag.yml inventory.yml isis.yml l2circuit.yml lacp.yml ldp.yml lldp.yml nd.yml ospf.yml pfestats.yml phyport.yml ppm.yml routes.yml securityzone.yml systemstorage.yml taskmemory.yml teddb.yml vlan.yml xcvr.yml


In [13]:
!cat ~/Works/Practice/DEVOPS/Documents/JNCIS/source/venv/lib/python3.9/site-packages/jnpr/junos/op/arp.yml

---
ArpTable:
  rpc: get-arp-table-information
  args:
     no-resolve: True
  item: arp-table-entry
  key: mac-address
  view: ArpView

ArpView:
  fields:
    mac_address: mac-address
    ip_address: ip-address
    interface_name: interface-name


The table contains the RPC call to use to get the information we want.<br>
The view contains the fields from the RPC call we are interested in.<br>

Let's now use the ARP table and view shown above to structure and process some data.<br>
We connect to a router using Junos PyEZ as normal, then we instantiate an object from the ArpTable class we've imported and issue the ```.get()``` method on it to populate it with data.<br>

We can then iterate over the object and view the data within it.


In [ ]:
from jnpr.junos import Device
from jnpr.junos.op.arp import ArpTable

# Define variables
UID = 'netconf'
PWD = 'netconf123'
ROUTER = '172.12.1.2'

# Instantiate a device object
with Device(host=ROUTER, password=PWD, user=UID, normalize=True) as dev:
    # instantiate a table object
    arp = ArpTable(dev)
    # Populate the object 
    arp.get()
    # Iterate over the items contained in the object 
    for mac in arp:
        print("{}: {}  {}".format(mac.mac_address, mac.ip_address, mac.interface_name))

: 

## Using Junos PyEZ to Push Configuration

Here we will use a small script to demonstrate how you can push configuration to Junos devices using Junos PyEZ.

You connect to the remote device as normal.  You then instantiate a configuration object from the Config() class.  Configuration (in several different formats e.g. set, text, xml, & jinja2) can then be applied.

Once the configuration has been applied you can check the config diff and commit it as you would if connected via the CLI.

When you are writing Junos PyEZ scripts you will need to write error handling to capture exceptions you could encounter. Without exception handling, errors could lead to your script exiting before it's complete.  For example, say you are pushing configuration to many devices and a few are down for maintainance.  You don't want the script to exit when you encounter one of the down routers.

We use a try/except block to capture 2 common connection errors and if these errors are encountered the router is skipped and we move onto the next.

Be sure to be explicit when you capture exceptions as you don't want to ignore other types of errors that have occured upstream.

In [ ]:
import sys
from jnpr.junos import Device
from jnpr.junos.utils.config import Config
from jnpr.junos.exception import (ConnectAuthError, ConnectRefusedError)

# Define variables
UID = 'netconf'
PWD = 'netconf123'
ROUTERS = ['172.12.1.1']
CONFIG="set system name-server 192.0.2.2"

# Iterate over the ROUTERS list
for router in ROUTERS:
    print("=" * 60)
    # Try / except block to capture 2 connectivity errors
    try:
        with Device(host=router, password=PWD, user=UID, normalize=True) as dev:
            print("connected to: {}\n".format(router))
            # Instantiate a config object
            with Config(dev) as conf_obj:
                # Load the changes into candidate configuration
                conf_obj.load(CONFIG, format='set')
                # Output the resulting configuration changes
                config_diff = conf_obj.diff().replace('+', '\n')
                print("The changes are: {}\n".format(config_diff))
                # Commit the changes
                conf_obj.commit()
                print("changes have been commited")
    except ConnectAuthError:
        print("Connection authentication error to: {}.  Skipping this device".format(router))
        continue
    except ConnectRefusedError:
        print("Connection refused to: {}.  Skipping this device".format(router))
        continue
    except Exception as message:
        print("Caught another exception, exiting.  It was:")
        print(message)
        sys.exit()
    